In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

/home/janek/.local/share/virtualenvs/rosetta-WKmHhL03/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
import keras.layers as L
from keras.models import Model
import numpy as np
import pandas as pd

RANDOM_STATE=42
np.random.seed(RANDOM_STATE)

In [14]:
START = '^'
END = '$'

SIZE = 1000
LATENT_DIM = 128
EMBEDDING_DIM = 32

In [4]:
def create_equations_df(size, min_value=0, max_value=9999, operations={'+': np.add, '-': np.subtract}):
    df = pd.DataFrame()
    df['a'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['b'] = np.random.randint(low=min_value, high=max_value, size=size)
    df['op'] = np.random.choice(list(operations.keys()), size)
    df['result'] = np.zeros(size, dtype='int')
    for symbol, calc in operations.items():
        df.loc[df.op == symbol, 'result'] = calc(df[df.op == symbol]['a'], df[df.op == symbol]['b'])
        
    df['input_texts'] = START + df.a.astype(str) + df.op + df.b.astype(str) + END
    df['target_texts'] = START + df.result.astype(str) + END
    return df

In [5]:
df = create_equations_df(SIZE)

In [6]:
corpus = pd.concat([df.input_texts, df.target_texts])

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=None, filters=None, char_level=True)
tokenizer.fit_on_texts(corpus)
df['input_sequences'] = tokenizer.texts_to_sequences(df.input_texts)
df['target_sequences'] = tokenizer.texts_to_sequences(df.target_texts)

In [8]:
X = keras.preprocessing.sequence.pad_sequences(df.input_sequences, padding='post').astype(np.float32)
# y = keras.preprocessing.sequence.pad_sequences(df.target_sequences)
y = df.result.astype(np.float32)

In [41]:
max_len = X.shape[1]
encoder_inputs = L.Input(shape=(max_len, ), dtype='float32', name='encoder_inputs')
embedding = L.Embedding(len(tokenizer.word_index), EMBEDDING_DIM, name='embedding')(encoder_inputs)
encoder_outputs, encoder_states = L.GRU(LATENT_DIM, return_state=True, name='encoder_gru')(embedding)
dense = L.Dense(1, activation='relu', name='dense')(encoder_states)

model = Model(inputs=encoder_inputs, outputs=dense)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_inputs (InputLayer)  (None, 11)                0         
_________________________________________________________________
embedding (Embedding)        (None, 11, 32)            448       
_________________________________________________________________
encoder_gru (GRU)            [(None, 128), (None, 128) 61824     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 62,401
Trainable params: 62,401
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X, y, validation_split=0.2, epochs=10, random_state=RANDOM_STATE)

Train on 80000 samples, validate on 20000 samples
Epoch 1/10
80000/80000 [==============================] - 22s 273us/step - loss: 65137532.0488 - val_loss: 63996375.2576
Epoch 2/10
80000/80000 [==============================] - 21s 267us/step - loss: 62232438.6248 - val_loss: 61228798.6656
Epoch 3/10
80000/80000 [==============================] - 21s 267us/step - loss: 59547733.1936 - val_loss: 58659887.1488
Epoch 4/10
80000/80000 [==============================] - 22s 275us/step - loss: 57079859.8016 - val_loss: 56305903.0720
Epoch 5/10
80000/80000 [==============================] - 22s 271us/step - loss: 54805000.7192 - val_loss: 54143027.5616
Epoch 6/10
80000/80000 [==============================] - 23s 285us/step - loss: 52723719.9040 - val_loss: 52171149.6064
Epoch 7/10
80000/80000 [==============================] - 23s 285us/step - loss: 50843493.9968 - val_loss: 50402347.0976
Epoch 8/10
80000/80000 [==============================] - 22s 273us/step - loss: 49161389.9344 - val_lo

In [37]:
model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(['^360+120$']), padding='post', maxlen=X.shape[1]))

array([[1957.2666]], dtype=float32)